STARTED:      2/26/25

LAST UPDATED: 2/26/25

By Eugene Fong

# GOAL(S)

- Pseudobulk on the SN data

# #* TODO

XYLENA: 
- pseudobulk each category (young/old) then run w/the tools Adam used
- avg's expression of each cell type in each sample
- should have a tool in scanpy
- also in Adam's pipeline

ADAM
- break up the adata obj into indiv clusters by cell type for the trajectory analysis
- redo the DGE by cell type, 
- use the leiden_2 cluster

Pseudo-bulking = collapsing into counts the mean expression for that sample
- sum is better for genes that are lowly expressed by highly variable betw samples
- mean is better for genes that are highly expressed genes

Lit review on pseudotime and trajectory analysis tools, run by some suggestions by Xylena before starting on one

# IMPORTS

In [ ]:
import numpy as np
import os
import pandas as pd
import anndata as ad
import scanpy as sc
import matplotlib.pyplot as plt
import seaborn as sns
import pprint

# LOAD DATA

### DEFINE PATHS

In [2]:
input_file = os.path.join('/data/CARD_singlecell/Brain_atlas/SN_Multiome/control_atlas/05_annotated_anndata_rna.h5ad')

In [3]:
# Define output file path for my testing, base on the original file name
output_file = os.path.join('output', os.path.basename(input_file).replace('.h5ad', '-ef-rank_genes_groups.h5ad'))
print('output_file =', output_file)

output_file = output/05_annotated_anndata_rna-ef-rank_genes_groups.h5ad


In [4]:
# # Save the AnnData object
# adata.write_h5ad(
#     filename = output_file, 
#     compression = 'gzip'
#     )

In [5]:
# RELOAD - my saved AnnData object
adata = sc.read_h5ad(output_file)
adata

AnnData object with n_obs × n_vars = 477356 × 35633
    obs: 'n_genes_by_counts', 'total_counts', 'total_counts_rb', 'pct_counts_rb', 'total_counts_mt', 'pct_counts_mt', 'doublet_score', 'cell_barcode', 'batch', 'sex', 'age', 'pmi', 'ethnicity', 'race', 'brain_bank', 'homogenization', 'library', 'seq', 'sample', 'short diagnosis', 'S_score', 'G2M_score', 'phase', 'n_counts', 'atlas_identifier', 'leiden_2', 'leiden', 'leiden_05', 'cell_type', 'age_old', 'age_bin', 'log1p_n_genes_by_counts', 'log1p_total_counts', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes', 'log1p_total_counts_mt', 'total_counts_ribo', 'log1p_total_counts_ribo', 'pct_counts_ribo'
    var: 'n_cells', 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'highly_variable_nbatches', 'highly_variable_intersection', 'mt', 'rb', 'ribo', 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_t

# DGE

### REF: [https://www.sc-best-practices.org/conditions/differential_gene_expression.html](https://www.sc-best-practices.org/conditions/differential_gene_expression.html)

# 16.2 - ENV SETUP

# INSTALLS

In [25]:
import os
os.environ['RPY2_CFFI_MODE'] = 'ABI'

# %pip install anndata2ri

In [ ]:
# %pip install pertpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 626.4/626.4 kB 10.0 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.4/40.4 kB 354.4 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.2/69.2 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.2/213.2 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.6/122.6 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.7/293.7 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.9/250.9 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━

#! WARNING - do this in terminal
#! WARNING - must use `module load rpy2` instead of installing it

```bash
import rpy2

python -m rpy2.situation

```

In [ ]:
# %pip install rpy2-rinterface

  Using cached rpy2_rinterface-3.6.0.dev2.tar.gz (75 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for rpy2-rinterface: filename=rpy2_rinterface-3.6.0.dev2-py3-none-any.whl size=94735 sha256=6939eb1916511428e2bedddc6d7a114e3738a69e2a56403c62b07f2c408d6694
  Stored in directory: /spin1/home/linux/fonge2/.cache/pip/wheels/95/9f/3e/c423e2e6934c7b5b6c1ce878fe1e7170eee2187188d6955992
Successfully built rpy2-rinterface
Note: you may need to restart the kernel to use updated packages.


In [ ]:
# %pip install rpy2-robjects

  Using cached rpy2_robjects-3.6.0.dev2-py3-none-any.whl.metadata (3.3 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.4/116.4 kB 7.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [29]:
%pip install rpy2.rinterface_lib.callbacks

ERROR: Could not find a version that satisfies the requirement rpy2.rinterface_lib.callbacks (from versions: none)
ERROR: No matching distribution found for rpy2.rinterface_lib.callbacks
Note: you may need to restart the kernel to use updated packages.


In [30]:

%pip install sc_toolbox
%pip install rpy2.robjects

Traceback (most recent call last):
  File "/data/fonge2/conda/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/data/fonge2/conda/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/data/fonge2/conda/lib/python3.10/site-packages/pip/__main__.py", line 22, in <module>
    from pip._internal.cli.main import main as _main
  File "/data/fonge2/conda/lib/python3.10/site-packages/pip/_internal/cli/main.py", line 10, in <module>
    from pip._internal.cli.autocompletion import autocomplete
  File "/data/fonge2/conda/lib/python3.10/site-packages/pip/_internal/cli/autocompletion.py", line 10, in <module>
    from pip._internal.cli.main_parser import create_main_parser
  File "/data/fonge2/conda/lib/python3.10/site-packages/pip/_internal/cli/main_parser.py", line 9, in <module>
    from pip._internal.build_env import get_runnable_pip
  File "/data/fonge2/conda/lib/python3.10/site-packages/pip/_inter

# IMPORTS

In [1]:
import warnings

warnings.filterwarnings("ignore")

import logging
import random

import anndata2ri
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pertpy
import rpy2.rinterface_lib.callbacks
import sc_toolbox
import scanpy as sc
import seaborn as sns
from rpy2.robjects import pandas2ri

sc.settings.verbosity = 0
rpy2.rinterface_lib.callbacks.logger.setLevel(logging.ERROR)

pandas2ri.activate()
anndata2ri.activate()

%load_ext rpy2.ipython

Unable to determine R home: [Errno 13] Permission denied: 'R'


ValueError: r_home is None. Try python -m rpy2.situation

# #! ERROR

r_home is None. Try python -m rpy2.situation

Unable to determine R home: [Errno 13] Permission denied: 'R'